# 5章 誤差逆伝播法

In [1]:
abstract type AbstractLayer
end

## 5.4 単純なレイヤの実装

### 5.4.1 乗算レイヤの実装

In [2]:
mutable struct MulLayer{T} <: AbstractLayer
    x::T
    y::T
end

(::Type{MulLayer{T}})() where {T} = MulLayer(zero(T), zero(T))

In [3]:
function forward(lyr::MulLayer{T}, x::T, y::T) where {T}
    lyr.x = x
    lyr.y = y
    x * y
end
@inline forward(lyr::MulLayer{T}, x, y) where {T} = forward(lyr, T(x), T(y))

forward (generic function with 2 methods)

In [4]:
function backward(lyr::MulLayer{T}, dout::T) where {T}
    dx = dout * lyr.y
    dy = dout * lyr.x
    (dx, dy)
end

backward (generic function with 1 method)

In [5]:
apple = 100;
apple_num = 2;
tax = 1.1;

In [6]:
# layer
mul_apple_layer = MulLayer{Float64}()
mul_tax_layer = MulLayer{Float64}()

MulLayer{Float64}(0.0, 0.0)

In [7]:
# forward
apple_price = forward(mul_apple_layer, apple, apple_num)
price = forward(mul_tax_layer, apple_price, tax)

220.00000000000003

In [8]:
# backward
dprice = 1.0
dapple_price, dtax = backward(mul_tax_layer, dprice)
dapple, dapple_num = backward(mul_apple_layer, dapple_price)
(dapple, dapple_num, dtax)

(2.2, 110.00000000000001, 200.0)

### 5.4.2 加算レイヤの実装

In [9]:
struct AddLayer <: AbstractLayer end

In [10]:
function forward(lyr::AddLayer, x, y)
    x + y
end

forward (generic function with 3 methods)

In [11]:
function backward(lyr::AddLayer, dout)
    (dout, dout)
end

backward (generic function with 2 methods)

In [12]:
apple = 100;
apple_num = 2;
orange = 150;
orange_num = 3;
tax = 1.1;

In [13]:
# layer
mul_apple_layer = MulLayer{Float64}()
mul_orange_layer = MulLayer{Float64}()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer{Float64}()

MulLayer{Float64}(0.0, 0.0)

In [14]:
# forward
apple_price = forward(mul_apple_layer, apple, apple_num)
orange_price = forward(mul_orange_layer, orange, orange_num)
all_price = forward(add_apple_orange_layer, apple_price, orange_price)
price = forward(mul_tax_layer, all_price, tax)

715.0000000000001

In [15]:
# backward
dprice = 1.0
dall_price, dtax = backward(mul_tax_layer, dprice)
dapple_price, dorange_price = backward(add_apple_orange_layer, dall_price)
dorange, dorange_num = backward(mul_orange_layer, dorange_price)
dapple, dapple_num = backward(mul_apple_layer, dapple_price)
(dapple, dapple_num, dorange, dorange_num, dtax)

(2.2, 110.00000000000001, 3.3000000000000003, 165.0, 650.0)

## 5.5 活性化関数レイヤの実装

### 5.5.1 ReLU レイヤ

In [16]:
mutable struct ReluLayer <: AbstractLayer
    mask::AbstractArray{Bool}
    ReluLayer() = new()
end

In [17]:
function forward(lyr::ReluLayer, x::AbstractArray{T}) where {T}
    mask = lyr.mask = (x .<= 0)
    out = copy(x)
    out[x .<= 0] .= zero(T)
    out
end

forward (generic function with 4 methods)

In [18]:
function backward(lyr::ReluLayer, dout::AbstractArray{T}) where {T}
    dout[lyr.mask] .= zero(T)
    dout
end

backward (generic function with 3 methods)

In [19]:
relulyr = ReluLayer()
forward(relulyr, [1.0 -0.5; -2.0 3.0])

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  3.0

In [20]:
relulyr.mask

2×2 BitArray{2}:
 false   true
  true  false

In [21]:
backward(relulyr, [1.0 1.0; 1.0 1.0])

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

### 5.5.2 Sigmoid レイヤ

In [22]:
mutable struct SigmoidLayer{T} <: AbstractLayer
    out::T
end
(::Type{SigmoidLayer{T}})() where {T} = SigmoidLayer(zero(T))

In [23]:
function forward(lyr::SigmoidLayer{T}, x::T) where {T}
    lyr.out = 1 ./ (1 .+ exp.(-x))
end

forward (generic function with 5 methods)

In [24]:
function backward(lyr::SigmoidLayer{T}, dout::T) where {T}
    dout .* (1 .- lyr.out) .* lyr.out
end

backward (generic function with 4 methods)

## 5.6 Affine/Softmax レイヤの実装

### 5.6.2 バッチ版 Affine レイヤ

In [25]:
mutable struct AffineLayer{T} <: AbstractLayer
    W::AbstractMatrix{T}
    b::AbstractVector{T}
    x::AbstractArray{T}
    dW::AbstractMatrix{T}
    db::AbstractVector{T}
    function (::Type{AffineLayer})(W::AbstractMatrix{T}, b::AbstractVector{T}) where {T}
        lyr = new{T}()
        lyr.W = W
        lyr.b = b
        lyr
    end
end

In [26]:
alyr = AffineLayer(randn(3,2), randn(3))

AffineLayer{Float64}([0.0724445 1.1644; 0.875635 -1.15573; -0.627673 -1.18389], [-0.838078, -0.743806, 1.26953], #undef, #undef, #undef)

In [27]:
function forward(lyr::AffineLayer{T}, x::AbstractArray{T}) where {T}
    lyr.x = x
    lyr.W * x .+ lyr.b
end

forward (generic function with 6 methods)

In [28]:
function backward(lyr::AffineLayer{T}, dout::AbstractArray{T}) where {T}
    dx = lyr.W' * dout
    lyr.dW = dout * lyr.x'
    lyr.db = _sumvec(dout)
    dx
end
@inline _sumvec(dout::AbstractVector{T}) where {T}= dout
@inline _sumvec(dout::AbstractMatrix{T}) where {T} = vec(mapslices(sum, dout, dims=2))
@inline _sumvec(dout::AbstractArray{T,N}) where {T,N} = vec(mapslices(sum, dout, dims=2:N))

_sumvec (generic function with 3 methods)

### 5.6.3 Softmax-with-Loss レイヤ

In [29]:
function softmax(a::AbstractVector{T}) where {T<:AbstractFloat}
    c = maximum(a)  # オーバーフロー対策
    exp_a = exp.(a .- c)
    exp_a ./ sum(exp_a)
end

function softmax(a::AbstractMatrix{T}) where {T<:AbstractFloat}
    mapslices(softmax, a, dims=1)
end

softmax (generic function with 2 methods)

In [30]:
function crossentropyerror(y::Vector, t::Vector)
    δ = 1e-7  # アンダーフロー対策
    # -sum(t .* log(y .+ δ))
    -(t ⋅ log.(y .+ δ))
end
function crossentropyerror(y::Matrix, t::Matrix)
    batch_size = size(y, 2)
    δ = 1e-7  # アンダーフロー対策
    -dot(t, log.(y .+ δ)) / batch_size
end
function crossentropyerror(y::Matrix, t::Vector)
    batch_size = size(y, 2)
    δ = 1e-7  # アンダーフロー対策
    -sum([log(y[t[i]+1, i]) for i=1:batch_size] .+ δ) / batch_size
end

crossentropyerror (generic function with 3 methods)

In [31]:
mutable struct SoftmaxWithLossLayer{T} <: AbstractLayer
    loss::T
    y::AbstractArray{T}
    t::AbstractArray{T}
    (::Type{SoftmaxWithLossLayer{T}})() where {T} = new{T}()
end

In [32]:
softmaxlyr = SoftmaxWithLossLayer{Float64}()

SoftmaxWithLossLayer{Float64}(1.0e-323, #undef, #undef)

In [33]:
function forward(lyr::SoftmaxWithLossLayer{T}, x::AbstractArray{T}, t::AbstractArray{T}) where {T}
    lyr.t = t
    y = lyr.y = softmax(x)
    lyr.loss = crossentropyerror(y, t)
end

forward (generic function with 7 methods)

In [34]:
function backward(lyr::SoftmaxWithLossLayer{T}, dout::T=1) where {T}
    dout .* _swlvec(lyr.y, lyr.t)
end
@inline _swlvec(y::AbstractArray{T}, t::AbstractVector{T}) where {T} = y .- t
@inline _swlvec(y::AbstractArray{T}, t::AbstractMatrix{T}) where {T} = (y .- t) / size(t)[2]

_swlvec (generic function with 2 methods)

## 5.7 誤差逆伝播法の実装

### 5.7.2 誤差逆伝播法に対応したニューラルネットワークの実装

In [35]:
struct TwoLayerNet{T}
    a1lyr::AffineLayer{T}
    relu1lyr::ReluLayer
    a2lyr::AffineLayer{T}
    softmaxlyr::SoftmaxWithLossLayer{T}
end

In [36]:
function (::Type{TwoLayerNet{T}})(input_size::Int, hidden_size::Int, output_size::Int,
        weight_init_std::Float64=0.01) where {T}
    W1 = weight_init_std .* randn(T, hidden_size, input_size)
    b1 = zeros(T, hidden_size)
    W2 = weight_init_std .* randn(T, output_size, hidden_size)
    b2 = zeros(T, output_size)
    a1lyr = AffineLayer(W1, b1)
    relu1lyr = ReluLayer()
    a2lyr = AffineLayer(W2, b2)
    softmaxlyr = SoftmaxWithLossLayer{T}()
    TwoLayerNet(a1lyr, relu1lyr, a2lyr, softmaxlyr)
end

In [37]:
function predict(net::TwoLayerNet{T}, x::AbstractArray{T}) where {T}
    a1 = forward(net.a1lyr, x)
    z1 = forward(net.relu1lyr, a1)
    a2 = forward(net.a2lyr, z1)
    a2
end

predict (generic function with 1 method)

In [38]:
function loss(net::TwoLayerNet{T}, x::AbstractArray{T}, t::AbstractArray{T}) where {T}
    y = predict(net, x)
    forward(net.softmaxlyr, y, t)
end

loss (generic function with 1 method)

In [39]:
using LinearAlgebra
function accuracy(net::TwoLayerNet{T}, x::AbstractArray{T}, t::AbstractArray{T}) where {T}
    y = vec(mapslices(indmax, predict(net, x), 1))
    if ndims(t) > 1 t = vec(mapslices(indmax, t, 1)) end
    mean(y .== t)
end

accuracy (generic function with 1 method)

In [40]:
struct TwoLayerNetGrads{T}
    W1::AbstractMatrix{T}
    b1::AbstractVector{T}
    W2::AbstractMatrix{T}
    b2::AbstractVector{T}
end

In [41]:
function gradient(net::TwoLayerNet{T}, x::AbstractArray{T}, t::AbstractArray{T}) where {T}
    # forward
    loss(net, x, t)

    # backward
    dout = one(T)
    dz2 = backward(net.softmaxlyr, dout)
    da2 = backward(net.a2lyr, dz2)
    dz1 = backward(net.relu1lyr, da2)
    da1 = backward(net.a1lyr, dz1)
    TwoLayerNetGrads(net.a1lyr.dW, net.a1lyr.db, net.a2lyr.dW, net.a2lyr.db)
end

gradient (generic function with 1 method)

In [42]:
function numerical_gradient(f, x::Vector)
    h = 1e-4 # 0.0001
    # (f(x+h) - f(x-h)) / 2h
    map(1:length(x)) do idx
        tmp_val = x[idx]
        # f(x+h)
        x[idx] += h
        fxh1 = f(x)
        # f(x-h)
        x[idx] -= 2h
        fxh2 = f(x)
        # restore
        x[idx] = tmp_val
        (fxh1 - fxh2) / 2h
    end
end
function numerical_gradient(f, x::AbstractArray{T,N}) where {T,N}
    h = 1e-4 # 0.0001
    # (f(x+h) - f(x-h)) / 2h
    reshape(map(1:length(x)) do idx
        tmp_val = x[idx]
        # f(x+h)
        x[idx] += h
        fxh1 = f(x)
        # f(x-h)
        x[idx] -= 2h
        fxh2 = f(x)
        # restore
        x[idx] = tmp_val
        (fxh1 - fxh2) / 2h
    end, size(x))
end

numerical_gradient (generic function with 2 methods)

In [43]:
function numerical_gradient(net::TwoLayerNet{T}, x::AbstractArray{T}, t::AbstractArray{T}) where {T}
    # W1
    dW1 = numerical_gradient(copy(net.a1lyr.W)) do W
        loss(TwoLayerNet(AffineLayer(W, net.a1lyr.b), net.relu1lyr, net.a2lyr, net.softmaxlyr), x, t)
    end
    # b1
    db1 = numerical_gradient(copy(net.a1lyr.b)) do b
        loss(TwoLayerNet(AffineLayer(net.a1lyr.W, b), net.relu1lyr, net.a2lyr, net.softmaxlyr), x, t)
    end
    # W2
    dW2 = numerical_gradient(copy(net.a2lyr.W)) do W
        loss(TwoLayerNet(net.a1lyr, net.relu1lyr, AffineLayer(W, net.a2lyr.b), net.softmaxlyr), x, t)
    end
    # b2
    db2 = numerical_gradient(copy(net.a2lyr.b)) do b
        loss(TwoLayerNet(net.a1lyr, net.relu1lyr, AffineLayer(net.a2lyr.W, b), net.softmaxlyr), x, t)
    end
    TwoLayerNetGrads(dW1, db1, dW2, db2)
end

numerical_gradient (generic function with 3 methods)

In [44]:
function applygradient!(net::TwoLayerNet{T}, grads::TwoLayerNetGrads{T}, learning_rate::T) where {T}
    net.a1lyr.W -= learning_rate .* grads.W1
    net.a1lyr.b -= learning_rate .* grads.b1
    net.a2lyr.W -= learning_rate .* grads.W2
    net.a2lyr.b -= learning_rate .* grads.b2
end

applygradient! (generic function with 1 method)

### 5.7.3 誤差逆伝播法の勾配確認

In [45]:
using Flux.Data.MNIST

In [46]:
x_train = MNIST.images(:train);
t_train = MNIST.labels(:train);

In [47]:
network = TwoLayerNet{Float64}(784, 50, 10)

TwoLayerNet{Float64}(AffineLayer{Float64}([-0.00976362 0.0176839 … 0.00311165 -0.00726129; 0.0102735 -0.00729893 … 0.000779849 -0.000722565; … ; -0.0175967 -0.00120475 … -0.00594092 -0.00326354; 0.0225002 0.000169073 … 0.0185359 0.0122442], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], #undef, #undef, #undef), ReluLayer(#undef), AffineLayer{Float64}([0.00122803 -0.0327538 … 0.0118966 0.0206251; -5.58326e-5 -0.011213 … -0.0135733 -3.23515e-5; … ; 0.0124154 -0.00842322 … -0.014723 0.000603491; 0.0120225 0.0119883 … 0.00894016 -0.00760404], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], #undef, #undef, #undef), SoftmaxWithLossLayer{Float64}(0.0, #undef, #undef))

In [48]:
x_batch = zeros(784, 3)
for i in 1:3
    x_batch[:, i] = reshape(float.(x_train[i]), (784,))
end
x_batch

784×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮            
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [49]:
function onehot(t::AbstractVector, l::AbstractVector)
    r = zeros(Int, length(l), length(t))
    for i = 1:length(t)
        r[findfirst(v -> v == t[i], l), i] = 1
    end
    r
end

onehot (generic function with 1 method)

In [50]:
function onehot(::Type{T}, t::AbstractVector, l::AbstractVector) where {T}
    r = zeros(T, length(l), length(t))
    for i = 1:length(t)
        r[findfirst(v -> v == t[i], l), i] = one(T)
    end
    r
end

onehot (generic function with 2 methods)

In [51]:
t_batch = onehot(Float64, t_train[1:3], 0:9)

10×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [52]:
predict(network, x_batch)

10×3 Array{Float64,2}:
 -0.00430016    0.00308441    0.00171515 
 -0.00461164    0.00245073   -0.0035271  
 -0.00137036   -0.00245529   -0.00540793 
  0.00165513    0.000379665  -0.000375624
  0.0058964     0.0131126     0.0117242  
 -0.00162097   -0.0091316     0.00073106 
 -0.00191912   -0.00381262   -0.00334591 
  0.000466597  -0.00131239    0.00410217 
 -0.00132146   -0.00649835   -0.000550968
 -0.00465242   -0.00654962   -0.00184464 

In [53]:
loss(network, x_batch, t_batch)

2.297556479574712

In [54]:
grad_numerical = numerical_gradient(network, x_batch, t_batch)

TwoLayerNetGrads{Float64}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.0049622, -0.00320214, -0.00156957, 0.00216676, -0.00111543, 0.0014883, 0.00556613, -0.00710022, -0.00245508, 0.0015508  …  -0.00281012, -0.00011787, 0.00228688, -0.00198267, 0.000641251, 0.00278914, -0.0011673, -0.00428951, -0.00458692, 0.0055206], [0.00172866 0.00210096 … -0.0311443 0.00177381; 0.00172157 0.00210031 … 0.00753442 0.00176701; … ; 0.00172682 0.00210723 … 0.00751083 0.00177244; 0.00172378 0.00210022 … 0.00750509 0.00176912], [-0.233253, 0.0998741, 0.0997559, 0.100119, -0.232239, -0.233603, 0.0997608, 0.100172, 0.0997846, 0.099629])

In [55]:
grad_backprop = gradient(network, x_batch, t_batch)

TwoLayerNetGrads{Float64}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.00496221, -0.00320215, -0.00156957, 0.00216676, -0.00111543, 0.00148831, 0.00556613, -0.00710022, -0.00245508, 0.0015508  …  -0.00281013, -0.000117871, 0.00228689, -0.00198267, 0.000641252, 0.00278914, -0.00116731, -0.00428952, -0.00458692, 0.00552061], [0.00172866 0.00210096 … -0.0311443 0.00177381; 0.00172157 0.00210031 … 0.00753443 0.00176701; … ; 0.00172683 0.00210723 … 0.00751084 0.00177244; 0.00172379 0.00210022 … 0.00750509 0.00176912], [-0.233253, 0.0998742, 0.099756, 0.100119, -0.232239, -0.233603, 0.0997609, 0.100172, 0.0997847, 0.0996291])

In [56]:
extrema(grad_numerical.W1)

(-0.008018726860292702, 0.008669786153436831)

In [57]:
extrema(grad_backprop.W1)

(-0.00801873483855888, 0.00866979477797113)

In [58]:
extrema(grad_numerical.b1)

(-0.008041172374628758, 0.008008012248872376)

In [59]:
extrema(grad_backprop.b1)

(-0.008041180358022197, 0.008008020259147307)

In [60]:
extrema(grad_numerical.W2)

(-0.06435376492142098, 0.02109523939752833)

In [61]:
extrema(grad_backprop.W2)

(-0.06435382900078339, 0.021095260371244397)

In [62]:
extrema(grad_numerical.b2)

(-0.2336028867078177, 0.1001719013604685)

In [63]:
extrema(grad_backprop.b2)

(-0.23360312107797312, 0.10017200090878199)